In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
train = pd.read_csv("../data/schemas/warm_up/TrainSet.csv", sep=",")
validation = pd.read_csv("../data/schemas/warm_up/ValidationSet.csv", sep=",")
test = pd.read_csv("../data/schemas/warm_up/TestSet.csv", sep=",")

In [3]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.4   # For feature selection

In [4]:
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[0]])) > THRESHOLD: #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])
    
if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
print(independent_variables)

['Hour_1', 'Hour_2', 'Hour_3', 'Occupant Count (people)_3', '12h Outdoor Drybulb Temperature (C)', '24h Outdoor Drybulb Temperature (C)', '12h Outdoor Relative Humidity (%)', '6h Direct Solar Radiation (W/m2)', '12h Direct Solar Radiation (W/m2)']


In [5]:
THRESHOLD = 0.5   # For feature selection

columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[0]])) > THRESHOLD: #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])
    
if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
print(independent_variables)

['12h Outdoor Drybulb Temperature (C)', '12h Outdoor Relative Humidity (%)', '6h Direct Solar Radiation (W/m2)']


Prepare the data

In [6]:
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

print(X_train_default.shape)
print(X_val_default.shape)
print(X_test_default.shape)

(576, 3)
(96, 3)
(96, 3)


In [7]:
# Function to prepare the data into batches that will be passed into the model

def create_sequences(input_data, output_data, timesteps_input, timesteps_output):
    sequences, targets = [], []
    for i in range(len(input_data) - timesteps_input - timesteps_output + 1):
        seq = input_data[i:i + timesteps_input]
        target = output_data[i + timesteps_input: i + timesteps_input + timesteps_output]
        sequences.append(seq)
        targets.append(target)

    return np.array(sequences), np.array(targets)

In [8]:
X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

print(f"X_train = {X_train.shape}, Y_train = {Y_train.shape}\n"
      f"X_val = {X_val.shape}, Y_val = {Y_val.shape}\n"
      f"X_test = {X_test.shape}, Y_test = {Y_test.shape}")

X_train = (505, 48, 3), Y_train = (505, 24, 2)
X_val = (25, 48, 3), Y_val = (25, 24, 2)
X_test = (25, 48, 3), Y_test = (25, 24, 2)


In [9]:
print(Y_train[:, :, 0])

[[0.59739428 0.49429981 0.38708765 ... 0.73547528 0.69513525 0.71729728]
 [0.49429981 0.38708765 0.28880037 ... 0.69513525 0.71729728 0.6633147 ]
 [0.38708765 0.28880037 0.2346659  ... 0.71729728 0.6633147  0.55539687]
 ...
 [0.47416969 0.39741721 0.28594384 ... 0.4397153  0.45619247 0.45482902]
 [0.39741721 0.28594384 0.16805223 ... 0.45619247 0.45482902 0.41836388]
 [0.28594384 0.16805223 0.12974031 ... 0.45482902 0.41836388 0.39162712]]


In [10]:
RMSE_model_1 = []
RMSE_model_2 = []

In [11]:
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras import layers, models

In [12]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='tanh')(flatten_layer)
dense2 = layers.Dense(60, activation='tanh')(dense1)
dense3 = layers.Dense(24, activation='tanh')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='tanh')(flatten_layer)
dense5 = layers.Dense(60, activation='tanh')(dense4)
dense6 = layers.Dense(24, activation='tanh')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 2s 67ms/step - loss: 0.6409 - output_variable1_loss: 0.3706 - output_variable2_loss: 0.2703 - val_loss: 0.3033 - val_output_variable1_loss: 0.1471 - val_output_variable2_loss: 0.1562
Epoch 2/100
4/4 [==============================] - 0s 13ms/step - loss: 0.3195 - output_variable1_loss: 0.1813 - output_variable2_loss: 0.1382 - val_loss: 0.2070 - val_output_variable1_loss: 0.0964 - val_output_variable2_loss: 0.1106
Epoch 3/100
4/4 [==============================] - 0s 12ms/step - loss: 0.2201 - output_variable1_loss: 0.1163 - output_variable2_loss: 0.1038 - val_loss: 0.1632 - val_output_variable1_loss: 0.0752 - val_output_variable2_loss: 0.0880
Epoch 4/100
4/4 [==============================] - 0s 12ms/step - loss: 0.1712 - output_variable1_loss: 0.0837 - output_variable2_loss: 0.0875 - val_loss: 0.1359 - val_output_variable1_loss: 0.0623 - val_output_variable2_loss: 0.0737
Epoch 5/100
4/4 [==============================] - 0s 12ms/step 

In [13]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='relu')(flatten_layer)
dense2 = layers.Dense(60, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='relu')(flatten_layer)
dense5 = layers.Dense(60, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 59ms/step - loss: 0.6272 - output_variable1_loss: 0.4289 - output_variable2_loss: 0.1984 - val_loss: 0.3893 - val_output_variable1_loss: 0.2172 - val_output_variable2_loss: 0.1721
Epoch 2/100
4/4 [==============================] - 0s 13ms/step - loss: 0.5268 - output_variable1_loss: 0.3713 - output_variable2_loss: 0.1555 - val_loss: 0.3469 - val_output_variable1_loss: 0.1943 - val_output_variable2_loss: 0.1526
Epoch 3/100
4/4 [==============================] - 0s 12ms/step - loss: 0.4697 - output_variable1_loss: 0.3318 - output_variable2_loss: 0.1379 - val_loss: 0.3181 - val_output_variable1_loss: 0.1832 - val_output_variable2_loss: 0.1349
Epoch 4/100
4/4 [==============================] - 0s 13ms/step - loss: 0.4250 - output_variable1_loss: 0.3012 - output_variable2_loss: 0.1238 - val_loss: 0.2889 - val_output_variable1_loss: 0.1676 - val_output_variable2_loss: 0.1213
Epoch 5/100
4/4 [==============================] - 0s 11ms/step 

In [14]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(240, activation='relu')(flatten_layer)
dense2 = layers.Dense(120, activation='relu')(dense1)
dense3 = layers.Dense(48, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(240, activation='relu')(flatten_layer)
dense5 = layers.Dense(120, activation='relu')(dense4)
dense6 = layers.Dense(48, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 62ms/step - loss: 0.6126 - output_variable1_loss: 0.3566 - output_variable2_loss: 0.2560 - val_loss: 0.3059 - val_output_variable1_loss: 0.1277 - val_output_variable2_loss: 0.1782
Epoch 2/100
4/4 [==============================] - 0s 15ms/step - loss: 0.3528 - output_variable1_loss: 0.1952 - output_variable2_loss: 0.1576 - val_loss: 0.2759 - val_output_variable1_loss: 0.1250 - val_output_variable2_loss: 0.1509
Epoch 3/100
4/4 [==============================] - 0s 15ms/step - loss: 0.2730 - output_variable1_loss: 0.1359 - output_variable2_loss: 0.1371 - val_loss: 0.2299 - val_output_variable1_loss: 0.0981 - val_output_variable2_loss: 0.1318
Epoch 4/100
4/4 [==============================] - 0s 15ms/step - loss: 0.2167 - output_variable1_loss: 0.0952 - output_variable2_loss: 0.1215 - val_loss: 0.1856 - val_output_variable1_loss: 0.0736 - val_output_variable2_loss: 0.1120
Epoch 5/100
4/4 [==============================] - 0s 15ms/step 

In [15]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(480, activation='relu')(flatten_layer)
dense2 = layers.Dense(240, activation='relu')(dense1)
dense3 = layers.Dense(120, activation='relu')(dense2)
dense9 = layers.Dense(48, activation='relu')(dense3)
output_variable1 = layers.Dense(24, name='output_variable1')(dense9)

# Dense layers for variable 2
dense4 = layers.Dense(480, activation='relu')(flatten_layer)
dense5 = layers.Dense(240, activation='relu')(dense4)
dense6 = layers.Dense(120, activation='relu')(dense5)
dense7 = layers.Dense(48, activation='relu')(dense6)
output_variable2 = layers.Dense(24, name='output_variable2')(dense7)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=200, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/200
4/4 [==============================] - 1s 65ms/step - loss: 0.5161 - output_variable1_loss: 0.3378 - output_variable2_loss: 0.1782 - val_loss: 0.2622 - val_output_variable1_loss: 0.1366 - val_output_variable2_loss: 0.1256
Epoch 2/200
4/4 [==============================] - 0s 16ms/step - loss: 0.3072 - output_variable1_loss: 0.1971 - output_variable2_loss: 0.1102 - val_loss: 0.2096 - val_output_variable1_loss: 0.1209 - val_output_variable2_loss: 0.0888
Epoch 3/200
4/4 [==============================] - 0s 15ms/step - loss: 0.2320 - output_variable1_loss: 0.1519 - output_variable2_loss: 0.0801 - val_loss: 0.1642 - val_output_variable1_loss: 0.0983 - val_output_variable2_loss: 0.0659
Epoch 4/200
4/4 [==============================] - 0s 15ms/step - loss: 0.1849 - output_variable1_loss: 0.1222 - output_variable2_loss: 0.0627 - val_loss: 0.1346 - val_output_variable1_loss: 0.0832 - val_output_variable2_loss: 0.0514
Epoch 5/200
4/4 [==============================] - 0s 14ms/step 

In [16]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(60, activation='relu')(flatten_layer)
dense2 = layers.Dense(48, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(60, activation='relu')(flatten_layer)
dense5 = layers.Dense(48, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 61ms/step - loss: 0.6286 - output_variable1_loss: 0.4301 - output_variable2_loss: 0.1985 - val_loss: 0.4037 - val_output_variable1_loss: 0.2041 - val_output_variable2_loss: 0.1996
Epoch 2/100
4/4 [==============================] - 0s 15ms/step - loss: 0.5336 - output_variable1_loss: 0.3527 - output_variable2_loss: 0.1809 - val_loss: 0.3634 - val_output_variable1_loss: 0.1749 - val_output_variable2_loss: 0.1886
Epoch 3/100
4/4 [==============================] - 0s 12ms/step - loss: 0.4779 - output_variable1_loss: 0.3068 - output_variable2_loss: 0.1710 - val_loss: 0.3366 - val_output_variable1_loss: 0.1591 - val_output_variable2_loss: 0.1776
Epoch 4/100
4/4 [==============================] - 0s 13ms/step - loss: 0.4355 - output_variable1_loss: 0.2739 - output_variable2_loss: 0.1616 - val_loss: 0.3267 - val_output_variable1_loss: 0.1599 - val_output_variable2_loss: 0.1668
Epoch 5/100
4/4 [==============================] - 0s 12ms/step 

In [17]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 8) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='relu')(flatten_layer)
dense2 = layers.Dense(60, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='relu')(flatten_layer)
dense5 = layers.Dense(60, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 60ms/step - loss: 0.7287 - output_variable1_loss: 0.4619 - output_variable2_loss: 0.2668 - val_loss: 0.3742 - val_output_variable1_loss: 0.1912 - val_output_variable2_loss: 0.1830
Epoch 2/100
4/4 [==============================] - 0s 16ms/step - loss: 0.4910 - output_variable1_loss: 0.3257 - output_variable2_loss: 0.1653 - val_loss: 0.3114 - val_output_variable1_loss: 0.1552 - val_output_variable2_loss: 0.1562
Epoch 3/100
4/4 [==============================] - 0s 14ms/step - loss: 0.3987 - output_variable1_loss: 0.2530 - output_variable2_loss: 0.1457 - val_loss: 0.2648 - val_output_variable1_loss: 0.1232 - val_output_variable2_loss: 0.1416
Epoch 4/100
4/4 [==============================] - 0s 14ms/step - loss: 0.3274 - output_variable1_loss: 0.1945 - output_variable2_loss: 0.1329 - val_loss: 0.2615 - val_output_variable1_loss: 0.1305 - val_output_variable2_loss: 0.1310
Epoch 5/100
4/4 [==============================] - 0s 13ms/step 

In [18]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 8) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='relu')(flatten_layer)
dense2 = layers.Dense(60, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='relu')(flatten_layer)
dense5 = layers.Dense(60, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=200, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/200
4/4 [==============================] - 1s 61ms/step - loss: 0.6722 - output_variable1_loss: 0.4467 - output_variable2_loss: 0.2254 - val_loss: 0.3955 - val_output_variable1_loss: 0.2045 - val_output_variable2_loss: 0.1910
Epoch 2/200
4/4 [==============================] - 0s 12ms/step - loss: 0.5251 - output_variable1_loss: 0.3551 - output_variable2_loss: 0.1700 - val_loss: 0.3486 - val_output_variable1_loss: 0.1805 - val_output_variable2_loss: 0.1681
Epoch 3/200
4/4 [==============================] - 0s 14ms/step - loss: 0.4723 - output_variable1_loss: 0.3182 - output_variable2_loss: 0.1541 - val_loss: 0.3137 - val_output_variable1_loss: 0.1637 - val_output_variable2_loss: 0.1500
Epoch 4/200
4/4 [==============================] - 0s 12ms/step - loss: 0.4157 - output_variable1_loss: 0.2772 - output_variable2_loss: 0.1385 - val_loss: 0.2892 - val_output_variable1_loss: 0.1538 - val_output_variable2_loss: 0.1354
Epoch 5/200
4/4 [==============================] - 0s 13ms/step 

In [19]:
tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(96, activation='relu')(flatten_layer)
dense2 = layers.Dense(48, activation='relu')(dense1)
output_variable1 = layers.Dense(24, name='output_variable1')(dense2)

# Dense layers for variable 2
dense4 = layers.Dense(96, activation='relu')(flatten_layer)
dense5 = layers.Dense(48, activation='relu')(dense4)
output_variable2 = layers.Dense(24, name='output_variable2')(dense5)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 2s 64ms/step - loss: 0.6327 - output_variable1_loss: 0.3914 - output_variable2_loss: 0.2413 - val_loss: 0.3398 - val_output_variable1_loss: 0.1669 - val_output_variable2_loss: 0.1729
Epoch 2/100
4/4 [==============================] - 0s 15ms/step - loss: 0.4267 - output_variable1_loss: 0.2724 - output_variable2_loss: 0.1543 - val_loss: 0.2678 - val_output_variable1_loss: 0.1313 - val_output_variable2_loss: 0.1365
Epoch 3/100
4/4 [==============================] - 0s 16ms/step - loss: 0.3273 - output_variable1_loss: 0.1998 - output_variable2_loss: 0.1274 - val_loss: 0.2404 - val_output_variable1_loss: 0.1234 - val_output_variable2_loss: 0.1170
Epoch 4/100
4/4 [==============================] - 0s 14ms/step - loss: 0.2630 - output_variable1_loss: 0.1498 - output_variable2_loss: 0.1132 - val_loss: 0.2138 - val_output_variable1_loss: 0.1110 - val_output_variable2_loss: 0.1029
Epoch 5/100
4/4 [==============================] - 0s 14ms/step 

In [20]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.5  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='relu')(flatten_layer)
dense2 = layers.Dense(60, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='relu')(flatten_layer)
dense5 = layers.Dense(60, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 2s 67ms/step - loss: 0.5931 - output_variable1_loss: 0.3881 - output_variable2_loss: 0.2050 - val_loss: 0.3788 - val_output_variable1_loss: 0.1853 - val_output_variable2_loss: 0.1935
Epoch 2/100
4/4 [==============================] - 0s 12ms/step - loss: 0.4901 - output_variable1_loss: 0.3173 - output_variable2_loss: 0.1728 - val_loss: 0.3434 - val_output_variable1_loss: 0.1746 - val_output_variable2_loss: 0.1688
Epoch 3/100
4/4 [==============================] - 0s 12ms/step - loss: 0.4370 - output_variable1_loss: 0.2820 - output_variable2_loss: 0.1551 - val_loss: 0.3100 - val_output_variable1_loss: 0.1612 - val_output_variable2_loss: 0.1487
Epoch 4/100
4/4 [==============================] - 0s 14ms/step - loss: 0.3895 - output_variable1_loss: 0.2490 - output_variable2_loss: 0.1405 - val_loss: 0.2758 - val_output_variable1_loss: 0.1396 - val_output_variable2_loss: 0.1363
Epoch 5/100
4/4 [==============================] - 0s 11ms/step 

In [21]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.45  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='relu')(flatten_layer)
dense2 = layers.Dense(60, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='relu')(flatten_layer)
dense5 = layers.Dense(60, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 2s 142ms/step - loss: 0.5811 - output_variable1_loss: 0.3478 - output_variable2_loss: 0.2333 - val_loss: 0.3769 - val_output_variable1_loss: 0.1748 - val_output_variable2_loss: 0.2021
Epoch 2/100
4/4 [==============================] - 0s 11ms/step - loss: 0.4571 - output_variable1_loss: 0.2773 - output_variable2_loss: 0.1798 - val_loss: 0.3485 - val_output_variable1_loss: 0.1620 - val_output_variable2_loss: 0.1865
Epoch 3/100
4/4 [==============================] - 0s 10ms/step - loss: 0.3951 - output_variable1_loss: 0.2286 - output_variable2_loss: 0.1665 - val_loss: 0.3059 - val_output_variable1_loss: 0.1346 - val_output_variable2_loss: 0.1713
Epoch 4/100
4/4 [==============================] - 0s 13ms/step - loss: 0.3468 - output_variable1_loss: 0.1921 - output_variable2_loss: 0.1548 - val_loss: 0.2730 - val_output_variable1_loss: 0.1141 - val_output_variable2_loss: 0.1589
Epoch 5/100
4/4 [==============================] - 0s 11ms/step

In [22]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.35  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='relu')(flatten_layer)
dense2 = layers.Dense(60, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='relu')(flatten_layer)
dense5 = layers.Dense(60, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 58ms/step - loss: 0.5958 - output_variable1_loss: 0.3768 - output_variable2_loss: 0.2190 - val_loss: 0.3522 - val_output_variable1_loss: 0.1892 - val_output_variable2_loss: 0.1630
Epoch 2/100
4/4 [==============================] - 0s 11ms/step - loss: 0.3841 - output_variable1_loss: 0.2443 - output_variable2_loss: 0.1398 - val_loss: 0.3106 - val_output_variable1_loss: 0.1684 - val_output_variable2_loss: 0.1422
Epoch 3/100
4/4 [==============================] - 0s 12ms/step - loss: 0.3282 - output_variable1_loss: 0.2072 - output_variable2_loss: 0.1210 - val_loss: 0.2786 - val_output_variable1_loss: 0.1601 - val_output_variable2_loss: 0.1185
Epoch 4/100
4/4 [==============================] - 0s 13ms/step - loss: 0.2911 - output_variable1_loss: 0.1876 - output_variable2_loss: 0.1036 - val_loss: 0.2364 - val_output_variable1_loss: 0.1303 - val_output_variable2_loss: 0.1061
Epoch 5/100
4/4 [==============================] - 0s 12ms/step 

In [23]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.3  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='relu')(flatten_layer)
dense2 = layers.Dense(60, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='relu')(flatten_layer)
dense5 = layers.Dense(60, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 60ms/step - loss: 0.8008 - output_variable1_loss: 0.5466 - output_variable2_loss: 0.2542 - val_loss: 0.3788 - val_output_variable1_loss: 0.1779 - val_output_variable2_loss: 0.2008
Epoch 2/100
4/4 [==============================] - 0s 12ms/step - loss: 0.3750 - output_variable1_loss: 0.1965 - output_variable2_loss: 0.1785 - val_loss: 0.2963 - val_output_variable1_loss: 0.1121 - val_output_variable2_loss: 0.1841
Epoch 3/100
4/4 [==============================] - 0s 13ms/step - loss: 0.2952 - output_variable1_loss: 0.1320 - output_variable2_loss: 0.1632 - val_loss: 0.2634 - val_output_variable1_loss: 0.1019 - val_output_variable2_loss: 0.1614
Epoch 4/100
4/4 [==============================] - 0s 13ms/step - loss: 0.2496 - output_variable1_loss: 0.1040 - output_variable2_loss: 0.1457 - val_loss: 0.2596 - val_output_variable1_loss: 0.1108 - val_output_variable2_loss: 0.1488
Epoch 5/100
4/4 [==============================] - 0s 13ms/step 

In [24]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.25  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='relu')(flatten_layer)
dense2 = layers.Dense(60, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='relu')(flatten_layer)
dense5 = layers.Dense(60, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=100, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
    

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/100
4/4 [==============================] - 1s 60ms/step - loss: 0.7108 - output_variable1_loss: 0.4838 - output_variable2_loss: 0.2270 - val_loss: 0.3661 - val_output_variable1_loss: 0.1831 - val_output_variable2_loss: 0.1829
Epoch 2/100
4/4 [==============================] - 0s 13ms/step - loss: 0.4559 - output_variable1_loss: 0.2949 - output_variable2_loss: 0.1610 - val_loss: 0.3062 - val_output_variable1_loss: 0.1712 - val_output_variable2_loss: 0.1349
Epoch 3/100
4/4 [==============================] - 0s 13ms/step - loss: 0.3805 - output_variable1_loss: 0.2539 - output_variable2_loss: 0.1266 - val_loss: 0.3001 - val_output_variable1_loss: 0.1720 - val_output_variable2_loss: 0.1281
Epoch 4/100
4/4 [==============================] - 0s 13ms/step - loss: 0.3461 - output_variable1_loss: 0.2333 - output_variable2_loss: 0.1128 - val_loss: 0.2642 - val_output_variable1_loss: 0.1525 - val_output_variable2_loss: 0.1118
Epoch 5/100
4/4 [==============================] - 0s 14ms/step 

In [25]:
print(RMSE_model_1)
print(RMSE_model_2)

[0.08445466084019204, 0.1007679278702331, 0.09709089471238957, 0.09577011348728143, 0.1293124560572587, 0.12792568406550195, 0.11424647339853389, 0.11241958450874995, 0.11293700349082143, 0.12710485903403268, 0.23076786903827118, 0.17324415146951172, 0.22083596423183952]
[0.07774525416242717, 0.06421376109242027, 0.0695069737585145, 0.07991839341473173, 0.09360693775316445, 0.07782032680157815, 0.07492404769023456, 0.09472468062860123, 0.07689329061479479, 0.046464575849433976, 0.04148182029867014, 0.040002934865351406, 0.04414813507879747]


In [26]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.4  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='relu')(flatten_layer)
dense2 = layers.Dense(60, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='relu')(flatten_layer)
dense5 = layers.Dense(60, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
"""    
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=150, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
"""

history = model.fit(X_train, y={"output_variable1": Y_train[:, :, 0], 
                                "output_variable2": Y_train[:, :, 1]}, epochs=200, verbose=1)

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/200
16/16 [==============================] - 1s 2ms/step - loss: 0.3490 - output_variable1_loss: 0.2057 - output_variable2_loss: 0.1432
Epoch 2/200
16/16 [==============================] - 0s 2ms/step - loss: 0.1852 - output_variable1_loss: 0.0922 - output_variable2_loss: 0.0929
Epoch 3/200
16/16 [==============================] - 0s 2ms/step - loss: 0.1155 - output_variable1_loss: 0.0487 - output_variable2_loss: 0.0668
Epoch 4/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0840 - output_variable1_loss: 0.0375 - output_variable2_loss: 0.0464
Epoch 5/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0642 - output_variable1_loss: 0.0336 - output_variable2_loss: 0.0306
Epoch 6/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0515 - output_variable1_loss: 0.0318 - output_variable2_loss: 0.0197
Epoch 7/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0436 - output_variable1_loss: 0.0302 - output_variable2_loss:

In [27]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.4  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD:  #or abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
independent_variables.remove(columns_to_predict[0])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='relu')(flatten_layer)
dense2 = layers.Dense(60, activation='relu')(dense1)
dense3 = layers.Dense(24, activation='relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='relu')(flatten_layer)
dense5 = layers.Dense(60, activation='relu')(dense4)
dense6 = layers.Dense(24, activation='relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
   
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=150, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
"""

history = model.fit(X_train, y={"output_variable1": Y_train[:, :, 0], 
                                "output_variable2": Y_train[:, :, 1]}, epochs=200, verbose=1)
""" 

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

Epoch 1/150
4/4 [==============================] - 1s 57ms/step - loss: 0.6547 - output_variable1_loss: 0.4133 - output_variable2_loss: 0.2415 - val_loss: 0.4151 - val_output_variable1_loss: 0.2086 - val_output_variable2_loss: 0.2066
Epoch 2/150
4/4 [==============================] - 0s 12ms/step - loss: 0.5219 - output_variable1_loss: 0.3360 - output_variable2_loss: 0.1859 - val_loss: 0.3618 - val_output_variable1_loss: 0.1779 - val_output_variable2_loss: 0.1839
Epoch 3/150
4/4 [==============================] - 0s 11ms/step - loss: 0.4532 - output_variable1_loss: 0.2862 - output_variable2_loss: 0.1670 - val_loss: 0.3287 - val_output_variable1_loss: 0.1630 - val_output_variable2_loss: 0.1657
Epoch 4/150
4/4 [==============================] - 0s 11ms/step - loss: 0.4068 - output_variable1_loss: 0.2559 - output_variable2_loss: 0.1509 - val_loss: 0.3071 - val_output_variable1_loss: 0.1570 - val_output_variable2_loss: 0.1502
Epoch 5/150
4/4 [==============================] - 0s 13ms/step 

In [32]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.4  # For feature selection
columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[
        0]])) > THRESHOLD and abs(train[column].corr(train[columns_to_predict[1]])) > THRESHOLD:
        independent_variables.append(column)

#[independent_variables.remove(i) for i in columns_to_predict]
for i in columns_to_predict:
    if i in independent_variables:
        independent_variables.remove(columns_to_predict[i])

if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
    
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

tscv = TimeSeriesSplit(n_splits = 4) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 6) #patience parameter to be adjusted

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# Permute layer
permute_layer = layers.Permute((2, 1))(input_layer)

# Flatten layer
flatten_layer = layers.Flatten()(permute_layer)

# Dense layers for variable 1
dense1 = layers.Dense(120, activation='leaky_relu')(flatten_layer)
dense2 = layers.Dense(60, activation='leaky_relu')(dense1)
dense3 = layers.Dense(24, activation='leaky_relu')(dense2)
output_variable1 = layers.Dense(24, name='output_variable1')(dense3)

# Dense layers for variable 2
dense4 = layers.Dense(120, activation='leaky_relu')(flatten_layer)
dense5 = layers.Dense(60, activation='leaky_relu')(dense4)
dense6 = layers.Dense(24, activation='leaky_relu')(dense5)
output_variable2 = layers.Dense(24, name='output_variable2')(dense6)

# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
) #types of optimizer, loss function to be adjusted
    
   
for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=150, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              # validation_split = 0.8,
              verbose=1,
              callbacks=callback
    )
"""

history = model.fit(X_train, y={"output_variable1": Y_train[:, :, 0], 
                                "output_variable2": Y_train[:, :, 1]}, epochs=200, verbose=1)
""" 

# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []
#print(len(eval_target_variables))

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)
    

rmse = 0
for i in range(len(predictions_1)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_1[i][0], Y_test[i, :, 0]))}')
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE = {rmse}")

RMSE_model_1.append(rmse)

rmse = 0
for i in range(len(predictions_2)):
    #print(f'pred={i}, num = {sqrt(mean_squared_error(predictions_2[i][0], Y_test[i, :, 1]))}')
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE = {rmse}")

RMSE_model_2.append(rmse)

print(independent_variables)

Epoch 1/150
4/4 [==============================] - 4s 61ms/step - loss: 0.7190 - output_variable1_loss: 0.4462 - output_variable2_loss: 0.2728 - val_loss: 0.3595 - val_output_variable1_loss: 0.1876 - val_output_variable2_loss: 0.1719
Epoch 2/150
4/4 [==============================] - 0s 11ms/step - loss: 0.4399 - output_variable1_loss: 0.2889 - output_variable2_loss: 0.1510 - val_loss: 0.2755 - val_output_variable1_loss: 0.1465 - val_output_variable2_loss: 0.1289
Epoch 3/150
4/4 [==============================] - 0s 12ms/step - loss: 0.3353 - output_variable1_loss: 0.2112 - output_variable2_loss: 0.1241 - val_loss: 0.2370 - val_output_variable1_loss: 0.1257 - val_output_variable2_loss: 0.1113
Epoch 4/150
4/4 [==============================] - 0s 12ms/step - loss: 0.2577 - output_variable1_loss: 0.1521 - output_variable2_loss: 0.1056 - val_loss: 0.2124 - val_output_variable1_loss: 0.1160 - val_output_variable2_loss: 0.0965
Epoch 5/150
4/4 [==============================] - 0s 11ms/step 